# Wed. 29th or Tues 28 of Nov

Mike and I met in my office. Looked at annual plots of inventories.
Then saw a big dip in annual data that will be missed on cencus data.
So, here we model annual data

In [1]:
import shutup
shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
# USDA_data_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

# Read

In [3]:
SoI = ["Alabama", "Arkansas", "California", 
       "Colorado", "Florida", "Georgia", 
       "Idaho", "Illinois", "Iowa", 
       "Kansas", "Kentucky", "Louisiana", 
       "Mississippi", "Missouri", "Montana", 
       "Nebraska", "New Mexico", "North Dakota", 
       "Oklahoma", "Oregon", "South Dakota", 
       "Tennessee", "Texas", "Virginia", 
       "Wyoming"]

abb_dict = pd.read_pickle(param_dir + "state_abbreviations.sav")
SoI_abb = []
for x in SoI:
    SoI_abb = SoI_abb + [abb_dict["full_2_abb"][x]]

In [4]:
county_id_name_fips = pd.read_csv(Min_data_base + "county_id_name_fips.csv")
county_id_name_fips.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

county_id_name_fips = county_id_name_fips[county_id_name_fips.state.isin(SoI_abb)].copy()

county_id_name_fips.sort_values(by=["state", "county"], inplace=True)

county_id_name_fips = rc.correct_Mins_FIPS(df=county_id_name_fips, col_="county")
county_id_name_fips.rename(columns={"county": "county_fips"}, inplace=True)

county_id_name_fips.reset_index(drop=True, inplace=True)
print (len(county_id_name_fips.state.unique()))
county_id_name_fips.head(2)

25


,county_fips,county_name,fips,state
0,01001,Autauga County,1001,AL
1,01003,Baldwin County,1003,AL


In [5]:
county_id_name_fips["state_fip"] = county_id_name_fips.county_fips.str.slice(0, 2)
county_id_name_fips.head(2)

,county_fips,county_name,fips,state,state_fip
0,01001,Autauga County,1001,AL,01
1,01003,Baldwin County,1003,AL,01


In [6]:
herbRatio = pd.read_csv(data_dir_base + "Supriya/Nov30_HerbRatio/herbRatio.csv")
herbRatio.dropna(how="any", inplace=True)
herbRatio = rc.correct_4digitFips(df=herbRatio, col_="county_fips")
herbRatio.head(2)

,county_fips,herb_avg,herb_std,pixel_count
0,21007,28.000000,15.000000,2.0
2,21031,19.666667,11.440668,3.0


In [15]:
NPP = pd.read_csv(Min_data_base + "statefips_annual_MODIS_NPP.csv")
NPP.rename(columns={"NPP": "modis_npp",
                    "statefips90m": "state_fip"}, 
           inplace=True)
NPP = rc.correct_3digitStateFips_Min(NPP, "state_fip")

NPP = NPP[NPP.state_fip.isin(county_id_name_fips.state_fip)]
NPP.reset_index(drop=True, inplace=True)

NPP.head(2)

,year,state_fip,modis_npp
0,2001,01,0.816597
1,2001,05,0.659446


In [8]:
# Rangeland area and Total area:
county_RA_and_TA_fraction = pd.read_csv(reOrganized_dir + "county_rangeland_and_totalarea_fraction.csv")
county_RA_and_TA_fraction.rename(columns={"fips_id": "county_fips"}, inplace=True)

county_RA_and_TA_fraction = rc.correct_Mins_FIPS(df=county_RA_and_TA_fraction, col_="county_fips")
L = len(county_RA_and_TA_fraction.county_fips.unique())
print ("number of counties are {}.".format(L))
print (county_RA_and_TA_fraction.shape)
county_RA_and_TA_fraction.head(2)

number of counties are 2379.
(2379, 4)


,county_fips,rangeland_acre,county_area_acre,rangeland_fraction
0,01003,13037.43,1060302.72,0.010000
1,01005,18.23,575781.12,0.000032


In [9]:
county_RA_and_TA_fraction["state_fip"] = county_RA_and_TA_fraction.county_fips.str.slice(0, 2)
county_RA_and_TA_fraction.head(2)

,county_fips,rangeland_acre,county_area_acre,rangeland_fraction,state_fip
0,01003,13037.43,1060302.72,0.010000,01
1,01005,18.23,575781.12,0.000032,01


In [13]:
state_RA = county_RA_and_TA_fraction[["state_fip", "rangeland_acre"]].groupby("state_fip").sum()
state_RA.reset_index(drop=False, inplace=True)
state_RA = state_RA[state_RA.state_fip.isin(county_id_name_fips.state_fip)]
state_RA.reset_index(drop=True, inplace=True)
state_RA.head(2)

,state_fip,rangeland_acre
0,01,31570.99
1,05,33578.77


In [17]:
print (len(NPP.state_fip.unique()))
print (len(state_RA.state_fip.unique()))

25
25


In [18]:
NPP.head(2)

,year,state_fip,modis_npp
0,2001,01,0.816597
1,2001,05,0.659446


In [19]:
state_RA.head(2)

,state_fip,rangeland_acre
0,01,31570.99
1,05,33578.77


In [20]:
state_annual_NPP_Ra = pd.merge(NPP, state_RA, on = ["state_fip"], how = "left")
state_annual_NPP_Ra.head(2)

,year,state_fip,modis_npp,rangeland_acre
0,2001,01,0.816597,31570.99
1,2001,05,0.659446,33578.77


In [21]:
shannon_annual = pd.read_csv(reOrganized_dir + "Beef_Cows_fromCATINV.csv")
shannon_annual = shannon_annual[shannon_annual.state.isin(SoI_abb)]
shannon_annual.head(2)

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AL,103000.0,106000.0,108000.0,111000.0,114000.0,104000.0,100000.0,96000.0,94000.0,...,680000.0,701000.0,681000.0,652000.0,673000.0,693000.0,714000.0,705000.0,696000.0,697000.0
3,AR,86000.0,85000.0,84000.0,83000.0,83000.0,61000.0,36000.0,38000.0,30000.0,...,919000.0,851000.0,862000.0,863000.0,893000.0,914000.0,924000.0,935000.0,915000.0,925000.0


In [33]:
wanted_years = np.arange(state_annual_NPP_Ra.year.min(), state_annual_NPP_Ra.year.max())
print (wanted_years[:3])
print (wanted_years[-3:])
cols_ = ["state"] + list(wanted_years.astype(str))
shannon_annual = shannon_annual[cols_]


shannon_annual = shannon_annual[shannon_annual.state.isin(list(county_id_name_fips.state.unique()))]
shannon_annual.sort_values(by=["state"], inplace=True)
shannon_annual.reset_index(drop=True, inplace=True)
shannon_annual.head(2)

[2001 2002 2003]
[2017 2018 2019]


,state,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AL,737000.0,750000.0,761000.0,732000.0,724000.0,686000.0,687000.0,667000.0,668000.0,669000.0,659000.0,680000.0,701000.0,681000.0,652000.0,673000.0,693000.0,714000.0,705000.0
1,AR,923000.0,927000.0,949000.0,982000.0,967000.0,899000.0,921000.0,943000.0,906000.0,937000.0,928000.0,919000.0,851000.0,862000.0,863000.0,893000.0,914000.0,924000.0,935000.0


In [42]:
shannon_annual.transpose.size()

AttributeError: 'function' object has no attribute 'size'